[52 文本预处理](https://www.bilibili.com/video/BV1Fo4y1Q79L?spm_id_from=333.999.0.0)

In [73]:
import collections
import re
from d2l import torch as d2l

### 文本预处理



- 序列数据往往存在多种形式，文本是其中常见的形式之一，例如一篇文章可以被简单地看作是一串单词序列，甚至是一串字符序列
- 将文本当做时序序列，将文本中的字或者字符、词当成样本，样本之间是存在时序信息的，因此文本是一个很长的时序序列
- 文本预处理的核心思想是如何将文本中的词转化成能够训练的样本


### 常见的文本预处理步骤



1. 读取数据集：将文本作为字符串加载到内存中



    - 将数据集读取到由多条文本行组成的列表中，其中每一条文本行都是一个字符串
    - 将非大小写字符全部变成空格（这虽然是一种有损的操作，但是能够使后续的操作变得更加简单）
去掉回车
    - 将所有字母全部变成小写




2. 词元化：将字符串拆分为词元（如单词和字符）



    - tokenize 是 NLP 中一个比较常见的操作：将一个句子或者是一段文字转化成 token（字符串、字符或者是词）
    - 将文本行列表（ lines ）作为输入，列表中的每个元素都是一个文本序列（比如一条文本行）
    - 将每个文本序列拆分成一个词元列表，词元（ token，英文中 token 一般有两种表示单元：一种是一个词作为一个基本单元，词相对来说，会让机器学习的模型更简单一点；一种是一个字符串作为一个基本单元，好处是样本数量比较少，坏处是还需要学习字符串的构成，字符串是如何由词构成的）是文本的基本单位
    - 中文的话会有所不同，因为在中文的段落中，词与词之间的间隔不是使用空格来进行间隔的，所以在中文中如果想使用词来表示 token 的话，还需要对其进行分词，分词相对来讲不是很容易
通过拆分，文本序列就被拆分成了许多 token 列表，这些列表要么是空，要么是有许多 token 在其中
    - 最后返回一个由词元列表组成的列表，其中每个词元都是一个字符串（ string ）




3. 建立词表，将拆分的词元映射到数字索引：将文本转换为数字索引序列，方便模型操作



    - 词元的类型是字符串，而模型需要的输入是数字（模型训练使用的都是 tensor ，而 tensor 都是基于下标的），因此这种类型不方便模型使用，所以需要构建一个字典，通常也叫做词汇表（vocabulary），用来将字符串类型的 token （要么是 word ，要么是 char ）映射到从 0 开始的数字索引中
    - 首先将训练集中所有的文档合并到一起，然后对它们的唯一词元进行统计，得到最终的统计结果 -- 语料（ corpus ）
    - 然后根据每个唯一词元的出现频率，为其分配一个数字索引，对于出现次数较少的词元，通常会被移除，以降低复杂性（min_freq：在 NLP 中，有很多词是不出现的，如果使用词的话，这些词可能在文本中就出现了几次，在这种情况下如果要进行训练的话可能比较困难，这里的 min_freq 指的是一个 token 在文本序列中出现的最少次数，如果少于这个数字的话，会将这些出现频率较低的 token 全部视为 “unknown”）
    - 语料库中不存在或者是已删除的任何词元都将映射到一个特定的未知词元 “<unk>” 
    - 还可以选择增加一个列表，用于保存保留下来的词元，比如填充词元（ “<pad>” ）；序列开始词元（ “<bos>” ）；序列结束词元（ “<eos>” ）


In [74]:
# 读取数据集
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  #@save
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    # 所有非字母全部替换为空格，所有字母小写
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


In [75]:
# token: 词元
# 这里将每一行转换为
def tokenize(line:list,token = 'word')->list:
    """将文本拆分我单词或者字符标记"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token =='char':
        return [list(line) for line in lines]
    else:
        print('错误：未知类型'+token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


词元的类型是字符串，而模型需要的输入是数字，因此这种类型不方便模型使用。 现在，让我们构建一个字典，通常也叫做词表（vocabulary）， 用来将字符串类型的词元映射到从\(0\)开始的数字索引中。 我们先将训练集中的所有文档合并在一起，对它们的唯一词元进行统计， 得到的统计结果称之为语料（corpus）。 然后根据每个唯一词元的出现频率，为其分配一个数字索引。 很少出现的词元通常被移除，这可以降低复杂性。 另外，语料库中不存在或已删除的任何词元都将映射到一个特定的未知词元“unk”。 我们可以选择增加一个列表，用于保存那些被保留的词元， 例如：填充词元（“pad”）； 序列开始词元（“bos”）； 序列结束词元（“eos”）

In [76]:
class Vocab:  
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        """_summary_
            tokens : 词元
            min_freq : 去除低频词汇，低于其频率的直接不要
            reserved_tokens : 用于标注句子的开始和结束
        """
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True)
        # 未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):  #@save
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

In [77]:
vocab = Vocab(tokens)
# 排名最后的10个
print(list(vocab.token_to_idx.items())[-10:],end='\n\n')
# 排名最前的10个
print(list(vocab.token_to_idx.items())[:10])

[('manhood', 4570), ('fragmentary', 4571), ('discord', 4572), ('culminating', 4573), ('discussed', 4574), ('cheerlessly', 4575), ('advancement', 4576), ('makers', 4577), ('casual', 4578), ('brittle', 4579)]

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [78]:
vocab.token_freqs[:10]

[('the', 2261),
 ('i', 1267),
 ('and', 1245),
 ('of', 1155),
 ('a', 816),
 ('to', 695),
 ('was', 552),
 ('in', 541),
 ('that', 443),
 ('my', 440)]

In [79]:
vocab = Vocab(tokens,min_freq=5)
# 排名最后的10个
print(list(vocab.token_to_idx.items())[-10:],end='\n\n')
# 排名最前的10个
print(list(vocab.token_to_idx.items())[:10])

[('nights', 815), ('clambering', 816), ('fingers', 817), ('edge', 818), ('habit', 819), ('museum', 820), ('burning', 821), ('killing', 822), ('eastward', 823), ('grass', 824)]

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [80]:
def load_corpus_time_machine(max_tokens=-1):  #@save
    """返回时光机器数据集的词元索引列表和词表"""
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)